In [2]:
import numpy as np
import cv2

pengklasifikasiwajah = cv2.CascadeClassifier ("Data/haarcascade_frontalface_default.xml")
citra = cv2.imread('Data/citra.jpg')
abuAbu = cv2.cvtColor(citra, cv2.COLOR_BGR2GRAY)
deteksi_wajah = pengklasifikasiwajah.detectMultiScale ( abuAbu, scaleFactor = 1.1, minNeighbors = 5)

for (x, y, w, h) in deteksi_wajah:
    cv2.rectangle(citra, (x,y), (x+w, y+h), (0, 255, 0), 2)
    cv2.imshow('Citra Wajah',citra)

In [10]:
import numpy as np
import cv2

pengklasifikasiwajah1 = cv2.CascadeClassifier ('Data/lbpcascade_frontalface.xml')
citra1 = cv2.imread('Data/citra.jpg')
abuAbu1 = cv2.cvtColor(citra1, cv2.COLOR_BGR2GRAY)
deteksi_wajah1 = pengklasifikasiwajah1.detectMultiScale ( abuAbu1, scaleFactor = 1.1, minNeighbors = 5)

for (x, y, w, h) in deteksi_wajah1:
    cv2.rectangle(citra, (x,y), (x+w, y+h), (0, 255, 0), 2)
    cv2.imshow('Citra Wajah',citra1)

In [5]:
import cv2
import numpy as np
from PIL import Image
import pickle
import os

names = []
filename = "names.pkl"
f = open(filename, 'wb')
pickle.dump(names,f)
f.close()

#os.mkdir('dataset')
#os.mkdir('trainer')

def sample():
    cam = cv2.VideoCapture(0)
    cam.set (3, 640)
    cam.set (4, 480)
    deteksi_wajah = cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')
    with open('names.pkl','rb') as f:
        names = pickle.load(f)
    
    nama = input('Masukan Nama: ')
    names.append(nama)
    id = names.index(nama)
    
    count = 0
    
    while(True):
        ret, img = cam.read()
        abuAbu = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        wajah = deteksi_wajah.detectMultiScale(gray, 1.3, 5)
        
        for (x,y,w,h) in wajah:
            cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
            count += 1
            cv2.imwrite("dataset/"+nama+"." + str(id) + '.' + str(count) +".jpg", abuAbu[y:y+h,x:x+w])
            cv2.imshow('image', img)
            
            k = cv2.waitKey(100) & 0xff
            
            if k == 27:
                break
            elif count >= 80:
                break
                
    with open('names.pkl', 'wb') as f:
        pickle.dump(names, f)
        
    cam.release()
    cv2.destroyAllWindows()

In [ ]:
def belajar():
    path = 'dataset'
    pendeteksi = cv2.face.LBPHFaceRecognizer_create()
    deteksiwajah = cv2.CascadeClassifier("data/haarcascade_frontalface_default.xml")
    
    def getImagesAndLabels(path):
        imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
        sampelwajah=[]
        ids = []

        for imagePath in imagePaths:
            PIL_img = Image.open(imagePath).convert('L')
            img_numpy = np.array(PIL_img,'uint8')
            
            id = int(os.path.split(imagePath)[-1].split(".")[1])
            wajah = deteksiwajah.detectMultiScale(img_numpy)
            
            for (x,y,w,h) in faces:
                sampelwajah.append(img_numpy[y:y+h,x:x+w])
                ids.append(id)
                
            return sampelwajah,ids
        
        wajah,ids = getImagesAndLabels(path)
        pendeteksi.train(wajah, np.array(ids))
        pendeteksi.write('trainer/trainer.yml')
        print("{0} Training Wajah. Keluar ProgramTraining".format(len(np.unique(ids))))

In [ ]:
def deteksi():
    pendeteksi = cv2.face.LBPHFaceRecognizer_create()
    pendeteksi.read('trainer/trainer.yml')
    cascadePath = ('data/haarcascade_frontalface_default.xml')
    faceCascade = cv2.CascadeClassifier(cascadePath);
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)
    cam.set(4, 480)
    minW = 0.1*cam.get(3)
    minH = 0.1*cam.get(4)
    
    with open('names.pkl', 'rb') as f:
        names = pickle.load(f)
    
    while True:
        ret, img =cam.read()
        abuAbu = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        wajah = faceCascade.detectMultiScale(abuAbu, scaleFactor = 1.2, minNeighbors = 5, minSize = (int(minW), int(minH)),)
        
        for(x,y,w,h) in wajah:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            id, confidence = pendeteksi.predict(gray[y:y+h,x:x+w])
            
            if (confidence < 100):
                id = names[id]
                confidence = " {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
                confidence = " {0}%".format(round(100 - confidence))
                cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
                cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0),1)
                
                cv2.imshow('camera',img)
                k = cv2.waitKey(10) & 0xff
                
                if k == 27:
                    break
                    
    print("\nKeluar Deteksi.")
    cam.release()
    cv2.destroyAllWindows()

In [ ]:
def main():
    while True:
        choice = int(input('''
        1 --> Daftar Wajah Baru
        2 --> Deteksi Wajah.
        3 --> Keluar
        Pilih:
        '''))
        
        if choice == 1:
            sample()
            belajar()
            
        elif choice == 2:
            deteksi()
        
        elif choice == 3:
            print('\nKeluar Program.')
            break